# 파이썬 기반의 AI를 위한 기초수학, 확률및통계

In [1]:
## 강봉주 
## bonjour.kang@gmail.com
##
## 모집단 분포 추론
##

## 모평균의 추정 및 검정

In [2]:
# 필요한 패키지
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import scipy
from scipy import linalg as la
import scipy.stats as ss

# 한글출력
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

scipy.__version__

'1.3.2'

In [3]:
# 분위수 구하기
X = ss.norm(0, 1)
alpha = np.array([0.1, 0.05, 0.01])
z_alpha_half = X.ppf(1-alpha/2)
z_alpha_half.round(3)

array([1.645, 1.96 , 2.576])

In [4]:
# 모분산이 알려져 있지 않은 경우: t 분포 적용
# 분위수 구하기
n = 10
X = ss.t(df=n-1)
alpha = np.array([0.1, 0.05, 0.01])
t_alpha_half = X.ppf(1-alpha/2)
t_alpha_half.round(3)

array([1.833, 2.262, 3.25 ])

In [5]:
# [BANK] 데이터 읽기
url = 'https://github.com/bong-ju-kang/kmu-mba-statistics-winter/raw/master/data/bank.zip'
df = pd.read_csv(url)
df.shape

(4521, 17)

In [6]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [7]:
# 변수 정의
x = df['balance']

# 평균
np.round(np.mean(x), 2)

1422.66

In [8]:
# 표본 표준편차
np.round(np.std(x, ddof=1), 2)

3009.64

In [9]:
#  표본평균 표준편차 정의: np.std(x, ddof=1)/np.sqrt(len(x))
scale = ss.sem(x)
np.round(scale, 2)

44.76

In [10]:
# 신뢰수준 정의
confidence = 1 - 0.05

# 신뢰 구간
confint = ss.t.interval(alpha=confidence, df=len(x)-1, loc=np.mean(x), scale=ss.sem(x))
np.round(confint, 2)

array([1334.9 , 1510.41])

In [11]:
#  모평균의 유의성 검정
null_mean = 1300
res = ss.ttest_1samp(x, popmean=null_mean)
res

Ttest_1sampResult(statistic=2.740300624371668, pvalue=0.006162411872873913)

In [12]:
# 양측 검정의 결과이므로 해당 p-value를 2로 나눈다.
(res.pvalue/2).round(4)

0.0031

## 모 평균의 비교

### 짝 비교

In [13]:
# 데이터 구성
url = 'https://github.com/bong-ju-kang/kmu-mba-statistics-winter/raw/master/data/oatbran.csv'
df = pd.read_csv(url)
df.head(3)

,ID,CORFLK,OATBRAN
0,1,4.61,3.84
1,2,6.42,5.57
2,3,5.40,5.85


In [14]:
# 단일 표본의 평균 비교
d = df['CORFLK'] - df['OATBRAN']
res = ss.ttest_1samp(d, popmean=0)
res

Ttest_1sampResult(statistic=3.3443552467926585, pvalue=0.005277682686223332)

In [15]:
# 양측 검정이므로 해당 p-value를 2로 나눈다.
(res.pvalue/2).round(4)

0.0026

### 2개의 독립 모집단 비교

In [16]:
# 데이터 구성
url = 'https://github.com/bong-ju-kang/kmu-mba-statistics-winter/raw/master/data/MDIS_2018_HFWS.txt'
df = pd.read_csv(url, header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,2018,10200261,P,1127.905175,NaN,2255.0,G2,1,2,G1,...,NaN,NaN,1.0,80.0,60,150,4.0,NaN,NaN,NaN
1,2018,10200371,P,2223.797699,NaN,4515.0,G1,1,4,G2,...,NaN,NaN,1.0,70.0,150,200,4.0,NaN,NaN,NaN
2,2018,10201191,P,701.129372,NaN,1424.0,G1,1,4,G2,...,NaN,NaN,1.0,58.0,300,500,3.0,NaN,NaN,NaN


In [17]:
# 가구주 연령대: 22번째 열
# 순자산; 109번째 열

# 40 대 순자산 데이터
x = df[df.iloc[:, 21] == 'G04'].iloc[:, 108]
np.round(x.mean(), 3)

39347.948

In [18]:
np.round(x.std(), 3)

63218.508

In [19]:
# 50 대 순자산 데이터
y = df[df.iloc[:, 21] == 'G05'].iloc[:, 108]
np.round(y.mean(), 3)

34450.749

In [20]:
np.round(y.std(), 3)

65620.641

In [21]:
# 순자산 차이에 대한 검정
res = ss.ttest_ind(x, y, equal_var=False)
res

Ttest_indResult(statistic=4.027404466323176, pvalue=5.6837625799562505e-05)

In [22]:
# 양측 검정이므로 해당 p-value / 2
(res.pvalue/2).round(3)

0.0

In [23]:
# 분산이 동일하다는 가정
res = ss.ttest_ind(x, y, equal_var=True)
res

Ttest_indResult(statistic=3.9875217708387836, pvalue=6.716389065729837e-05)

## 모분산의 추정 및 검정

### 모분산 신뢰 구간

In [24]:
# [HWSF] 데이터의 순자산
x = df.iloc[:, 108]

np.round(np.var(x), 3)

3205003390.167

In [25]:
# 모분산 신뢰 구간

# 유의수준 정의
alpha = 0.05

# 표본 크기 정의
n = len(x)

# 표본 분산 정의
s2 = np.var(x, ddof=1)

upper = (n - 1) * s2 / ss.chi2.ppf(alpha / 2, n-1)
lower = (n - 1) * s2 / ss.chi2.ppf(1 - alpha / 2, n-1)
# (lower, upper)
np.round([lower, upper], 3)

array([3.14108451e+09, 3.27125730e+09])

### 모 분산비의 추정

In [26]:
# 데이터 구성
# 가구주 연령대: 22번째 열
# 순자산; 109번째 열

# 40 대 순자산 데이터
x = df[df.iloc[:, 21] == 'G04'].iloc[:, 108]

# 50 대 순자산 데이터
y = df[df.iloc[:, 21] == 'G05'].iloc[:, 108]

In [27]:
# 유의 수준 정의
alpha = 0.05

# 표본분산 비 정의
ratio = np.var(x, ddof=1)/np.var(y, ddof=2)
np.round(ratio, 3)

0.928

In [28]:
# F 분위수 정의
Falpha = ss.f.ppf(alpha/2, len(x)-1, len(y)-1)
F1minusAlpha= ss.f.ppf(1-alpha/2, len(x)-1, len(y)-1)

# 100(1-alpha)% 신뢰 구간
np.round([1/F1minusAlpha*ratio, 1/Falpha*ratio], 3)

array([0.881, 0.978])

### 모분산 비에 대한 검정

In [29]:
# 유의 수준 정의
alpha = 0.05

# p-value 계산: 양측 검정
pvalue = ss.f.cdf(ratio, len(x)-1, len(y)-1)/2
pvalue.round(4)

0.0014